In [4]:
import numpy as np
import re


diccionario_palabras = {}
diccionario_indices = {}
diccionario_onehot = {}
diccionario_onehot_a_palabra = {}

with open("/content/drive/MyDrive/Aprendizaje Automatico Avanzado/corpus.txt", "r", encoding="utf-8") as f:
    words = f.read().splitlines()

for token in words:
    if token not in diccionario_palabras:

        index = len(diccionario_palabras)
        diccionario_palabras[token] = index
        diccionario_indices[index] = token

cardinal_V = len(diccionario_palabras)

for token, idx in list(diccionario_palabras.items()):

    one_hot_vector = np.zeros(cardinal_V)
    one_hot_vector[idx] = 1
    diccionario_onehot[token] = one_hot_vector
    diccionario_onehot_a_palabra[str(one_hot_vector)] = token

In [20]:
def cargar_modelo_completo(nombre_archivo='pesos_cbow_pc2_epoca0.npz'):
    """
    Carga los pesos W1, W2 y los hiperparámetros N, C y eta
    desde un archivo .npz.
    """
    try:
        data = np.load(nombre_archivo)

        W1 = data['W1']
        W2 = data['W2']

        N = data['N'].item()
        C = data['C'].item()
        eta = data['eta'].item()

        print()

        return W1, W2, N, C, eta

    except FileNotFoundError:
        print(f"Error: No se encontró el archivo '{nombre_archivo}'.")
        return None, None, None, None, None
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [6]:
W1, W2, N, C, eta = cargar_modelo_completo("/content/drive/MyDrive/Aprendizaje Automatico Avanzado/pesos_cbow_pc2_epoca0.npz")

## Predicciones Reales: Donde se propaga como se entrena y se evalua si son, estan las predicciones y los accuracy estrictos y mas variados.

In [21]:
def predecirCBOW(diccionario_palabras, diccionario_indices, W1, W2, palabras_contexto, MasParecido=None):
    # Convertir contexto a índices
    contexto = [diccionario_palabras[token] for token in re.findall(r"\w+|[.,!?;:]", palabras_contexto) if token in diccionario_palabras]

    if not contexto:
        print("Las palabras no pertenecen al diccionario.")
        return None, None, None

    h = np.mean(W1[contexto], axis=0).reshape(-1, 1)

    u = W2.T @ h
    y = softmax(u)

    if MasParecido is not None and MasParecido > 0:
        indices_mas_grandes = np.argsort(y.flatten())[-MasParecido:][::-1]
        palabras_resultantes = [diccionario_indices[idx] for idx in indices_mas_grandes]
        valores = [y[idx] for idx in indices_mas_grandes]
        return indices_mas_grandes, palabras_resultantes, valores
    else:
        indice_mas_grande = np.argmax(y)
        palabra_resultante = diccionario_indices[indice_mas_grande]
        valor = y[indice_mas_grande]
        return indice_mas_grande, palabra_resultante, valor

In [23]:
predecirCBOW(diccionario_palabras, diccionario_indices, W1, W2, palabras_contexto="encontraria a la maga? tantas veces me habia bastado asomarme", MasParecido=5)

(array([10512, 16271, 19950,  7579,  3275]),
 ['referente', 'afueras', 'apresurarse', 'montellier', 'analogías'],
 [array([0.00854287]),
  array([0.00495958]),
  array([0.00427584]),
  array([0.00424342]),
  array([0.00390622])])

In [25]:
indices_top_5, palabras_top_5, h_vector_top_5 = predecirCBOW(diccionario_palabras, diccionario_indices, W1, W2, palabras_contexto="encontraría a la maga? tantas veces me había bastado asomarme", MasParecido=5)
indice_top_1, palabra_top_1, h_vector_top_1 = predecirCBOW(diccionario_palabras, diccionario_indices, W1, W2, palabras_contexto="encontraría a la maga? tantas veces me había bastado asomarme")

print(f"Top 5 palabras: {palabras_top_5}, con valores {h_vector_top_5}")
print(f"Mejor : {palabra_top_1}, con valores {h_vector_top_1}")

Top 5 palabras: ['referente', 'montellier', 'apresurarse', 'afueras', 'limpios'], con valores [array([0.00444491]), array([0.00311397]), array([0.00249189]), array([0.00249071]), array([0.00236007])]
Mejor : referente, con valores [0.00444491]


In [26]:
def predecirSkipGram(diccionario_palabras, diccionario_indices, W1, W2, palabra_central, contexto=3):
    if palabra_central not in diccionario_palabras:
        print("No está la palabra en el vocabulario")
        return None, None, None

    indice = diccionario_palabras[palabra_central]

    # Embedding de la palabra central
    h = W1[indice].reshape(-1, 1)

    # Predicción de contexto
    u = W2.T @ h
    y = softmax(u)

    indices_mas_grandes = np.argsort(y.flatten())[-contexto:][::-1]
    palabras_resultantes = [diccionario_indices[idx] for idx in indices_mas_grandes]
    valores = [y[idx] for idx in indices_mas_grandes]
    return indices_mas_grandes, palabras_resultantes, valores

In [28]:
indices_top_5, palabras_top_5, h_vector_top_5 = predecirSkipGram(diccionario_palabras, diccionario_indices, W1, W2,"había", contexto=3)
indice_top_1, palabra_top_1, h_vector_top_1 = predecirSkipGram(diccionario_palabras, diccionario_indices, W1, W2,"había", contexto=10)

print(f"Top 3 words: {palabras_top_5}, con valores {h_vector_top_5}")
print(f"Top 10 words: {palabra_top_1}, con valores {h_vector_top_1}")

Top 3 words: ['a', 'la', 'sufriera'], con valores [array([0.00033707]), array([0.00023499]), array([0.00023473])]
Top 10 words: ['a', 'la', 'sufriera', 'maga', 'tarde', 'num', 'montellier', 'fundada', 'salgo', 'limpios'], con valores [array([0.00033707]), array([0.00023499]), array([0.00023473]), array([0.00018261]), array([0.00018179]), array([0.00018029]), array([0.00017964]), array([0.0001571]), array([0.00013264]), array([0.00011812])]


In [29]:
def calcular_error_CBOW(diccionario_palabras, diccionario_indices, W1, W2, corpus, contexto=3, parecidos=3):
    indices = [i for i in range(contexto, (len(corpus) - contexto))]
    indices_contexto = [i for i in range(-contexto, 0)] + [i for i in range(1, contexto + 1)]
    indices_tuplas = [(diccionario_palabras[corpus[i]], [diccionario_palabras[corpus[i+j]] for j in indices_contexto]) for i in indices]

    aciertos_estrictos = 0
    aciertos_variados = 0

    for indice_central, contexto_indices in indices_tuplas:
        palabras_contexto = [diccionario_indices[j] for j in contexto_indices]

        # Top 1
        indice_top_1, _, _ = predecirCBOW(diccionario_palabras, diccionario_indices, W1, W2, " ".join(palabras_contexto), MasParecido=None)
        if indice_top_1 == indice_central:
            aciertos_estrictos += 1

        # Top k
        indices_top_k, _, _ = predecirCBOW(diccionario_palabras, diccionario_indices, W1, W2, " ".join(palabras_contexto), MasParecido=parecidos)
        if indice_central in indices_top_k:
            aciertos_variados += 1

    accuracy_estricto = aciertos_estrictos / len(indices_tuplas)
    accuracy_variado = aciertos_variados / len(indices_tuplas)
    return accuracy_estricto, accuracy_variado

In [ ]:
calcular_error_CBOW(diccionario_palabras, diccionario_indices, W1, W2, corpus=words, contexto=3, parecidos=3)

In [ ]:
def calcular_error_skipgram(diccionario_palabras, diccionario_indices, W1, W2, corpus, contexto=3):
    indices = [i for i in range(contexto, (len(corpus) - contexto))]
    indices_contexto = [i for i in range(-contexto, 0)] + [i for i in range(1, contexto + 1)]
    indices_tuplas = [(diccionario_palabras[corpus[i]], [diccionario_palabras[corpus[i+j]] for j in indices_contexto]) for i in indices]

    aciertos_estrictos = 0
    aciertos_variados = 0

    for indice_central, contexto_indices in indices_tuplas:
        palabra_central = diccionario_indices[indice_central]
        palabras_contexto = [diccionario_indices[j] for j in contexto_indices]

        _, palabras_top_3, _ = predecirSkipGram(diccionario_palabras, diccionario_indices, W1, W2, palabra_central, contexto=3)

        _, palabras_top_10, _ = predecirSkipGram(diccionario_palabras, diccionario_indices, W1, W2, palabra_central, contexto=10)

        if all(palabra in palabras_top_3 for palabra in palabras_contexto):
            aciertos_estrictos += 1

        if any(palabra in palabras_top_10 for palabra in palabras_contexto):
            aciertos_variados += 1

    accuracy_estricto = aciertos_estrictos / len(indices_tuplas)
    accuracy_variado = aciertos_variados / len(indices_tuplas)
    return accuracy_estricto, accuracy_variado

In [ ]:
calcular_error_skipgram(diccionario_palabras, diccionario_indices, W1, words, contexto=3)

## Accuracy mas aproximados a los W1, ergo los embeddings

In [ ]:
def predecirCBOW(diccionario_palabras, diccionario_indices, W1, corpus, palabras_contexto, MasParecido=None):
  contexto = [diccionario_palabras[i] for i in re.findall(r"\w+|[.,!?;:]", palabras_contexto)if i in diccionario_palabras]

  if not contexto:
    print("Las palabras no pertenecen al diccionario.")

  return None, None, None

  h = np.mean(W1[contexto], axis=0).reshape(-1, 1)

  if MasParecido is not None and MasParecido > 0:
     indices_mas_grandes = np.argsort(h.flatten())[-MasParecido:][::-1]
     palabras_resultantes = [diccionario_indices[indice] for indice in indices_mas_grandes]
     valores = [h[i] for i in indices_mas_grandes]
     return indices_mas_grandes, palabras_resultantes,valores
  else:
     indice_mas_grande = np.argmax(h)
     palabra_resultante = diccionario_indices[indice_mas_grande]
     valores = h[indice_mas_grande]
     return indice_mas_grande, palabra_resultante,valores

def predecirSkipGram(diccionario_palabras, diccionario_indices, W1, corpus,palabras, contexto=3):

  if palabras not in diccionario_palabras:

    print("No esta la palabra en el vocabulario")

  indice = diccionario_palabras[palabras]
  h = W1[indice]
  indices_mas_grandes = np.argsort(h.flatten())[-contexto:][::-1]
  palabras_resultantes = [diccionario_indices[indice] for indice in indices_mas_grandes]
  valores = [h[i] for i in indices_mas_grandes]

  return indices_mas_grandes, palabras_resultantes, valores

def calcular_error_CBOW(diccionario_palabras, diccionario_indices, W1, corpus, contexto=3, parecidos=3):
  indices = [i for i in range(contexto,(len(corpus)-contexto))]
  indices_contexto = [i for i in range(-contexto,0)] + [i for i in range(1,contexto+1)]
  indices_tuplas = [(diccionario_palabras[corpus[i]], [diccionario_palabras[corpus[i+j]] for j in indices_contexto]) for i in indices]
  aciertos_estrictos = 0
  aciertos_variados = 0

  for i, (indice,contexto)in enumerate(indices_tuplas): palabras_contexto = [diccionario_indices[j] for j in contexto]

  indice_top_1, palabra_top_1, h_vector_top_1 = predecirCBOW(diccionario_palabras, diccionario_indices, W1, words, " ".join(palabras_contexto), MasParecido=None)

  if indice_top_1 == indice:
     aciertos_estrictos += 1
  indice_top_varias, palabra_top_varias, h_vector_top_varias = predecirCBOW(diccionario_palabras, diccionario_indices, W1, words, " ".join(palabras_contexto), MasParecido=parecidos)

  if indice in indice_top_varias:
     aciertos_variados += 1
  accuracy_estricto = aciertos_estrictos / len(indices_tuplas)
  accuracy_variado = aciertos_variados / len(indices_tuplas)

  return accuracy_estricto, accuracy_variado

def calcular_error_skipgram(diccionario_palabras, diccionario_indices, W1, corpus, contexto=3):
   indices = [i for i in range(contexto,(len(corpus)-contexto))]
   indices_contexto = [i for i in range(-contexto,0)] + [i for i in range(1,contexto+1)]
   indices_tuplas = [(diccionario_palabras[corpus[i]], [diccionario_palabras[corpus[i+j]] for j in indices_contexto]) for i in indices]

   aciertos_estrictos = 0
   aciertos_variados = 0

   for i, (indice,contexto)in enumerate(indices_tuplas):
     palabra = diccionario_indices[indice]
     palabras_contexto_indices = contexto
     palabras_contexto = [diccionario_indices[j] for j in palabras_contexto_indices]
     indices_top_3, palabras_top_3, h_vector_top_3 = predecirSkipGram(diccionario_palabras, diccionario_indices, W1, words,palabra, contexto=3)
     indices_top_10, palabra_top_10, h_vector_top_10 = predecirSkipGram(diccionario_palabras, diccionario_indices, W1, words,palabra, contexto=10)

     if all(palabra_contexto in palabras_top_3 for palabra_contexto in palabras_contexto):
      aciertos_estrictos += 1

     if any(palabra_contexto in palabra_top_10 for palabra_contexto in palabras_contexto):
      aciertos_variados += 1

     accuracy_estricto = aciertos_estrictos / len(indices_tuplas)
     accuracy_variado = aciertos_variados / len(indices_tuplas)


     return accuracy_estricto, accuracy_variado